In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/selectedmitbih/resized_data.h5
/kaggle/input/selectedmitbih/test_dataset.h5


In [2]:
import h5py
import numpy as np

In [3]:
with h5py.File('/kaggle/input/selectedmitbih/resized_data.h5', 'r') as file:
    train_data = file['resized_dataset']
    train_labels = file['beat_type']
    train_data = np.array(train_data)
    train_labels = np.array(train_labels)


In [4]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
import numpy as np
from tensorflow.keras.optimizers import Adam
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


2024-03-29 06:36:18.023326: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 06:36:18.023435: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 06:36:18.173375: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
ros = RandomOverSampler()
train_data, train_labels = ros.fit_resample(train_data.reshape(-1, 224*224*3), train_labels)
train_data = train_data.reshape(-1, 224, 224, 3)

# Split the data into train and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# Define the number of classes
num_classes = 3
train_labels = to_categorical(train_labels, num_classes=num_classes)
val_labels =  to_categorical(val_labels, num_classes=num_classes)

# Load pre-trained ResNet50 model with weights from ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = True

# Add new fully-connected layers
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)  # Global average pooling to reduce dimensions
x = Dense(1024, activation='relu')(x)  # Add new fully-connected layer
x = Dense(512, activation='relu')(x)  # Add another fully-connected layer
predictions = Dense(num_classes, activation='softmax')(x)  # Add softmax layer for classification

model = Model(inputs=base_model.input, outputs=predictions)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
with h5py.File('/kaggle/input/selectedmitbih/test_dataset.h5', 'r') as file:
    test_data = file['resized_dataset']
    test_labels = file['beat_type']
    test_data = np.array(test_data)
    test_labels = np.array(test_labels)
    
test_labels = to_categorical(test_labels, num_classes=3)


In [7]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam, SGD
from sklearn.metrics import classification_report
import os

# Define a list of learning rates and optimizers to try
learning_rates = [0.001, 0.01, 0.1]
optimizers = [Adam, SGD]

best_model_path = 'best_model.keras'
best_accuracy = 0
best_hyperparameters = {}

for lr in learning_rates:
    for optimizer_class in optimizers:
        # Instantiate the optimizer with the current learning rate
        optimizer = optimizer_class(learning_rate=lr)
        
        # Compile the model with the optimizer and loss function
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        # Add callbacks for learning rate reduction, early stopping, and model checkpoint
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)
        early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
        checkpoint = ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True, mode='min')
        
        # Fit the data to the model
        history = model.fit(train_data, train_labels, batch_size=32, epochs=40, 
                            validation_data=(val_data, val_labels), 
                            callbacks=[reduce_lr, early_stop, checkpoint])
        
        # Evaluate the model on validation data
        val_loss, _ = model.evaluate(val_data, val_labels)
        
        predictions = model.predict(test_data)
        predicted_labels = tf.argmax(predictions, axis=1)
        true_labels = tf.argmax(test_labels, axis=1)  # Assuming you have true labels for comparison
        print('prediction complete')

        # Calculate evaluation metrics
        report = classification_report(true_labels, predicted_labels, output_dict=True)

        f1_score = report['weighted avg']['f1-score']
        precision = report['weighted avg']['precision']
        positive_predictive_value = report['weighted avg']['precision']
        specificity = report['weighted avg']['recall']
        sensitivity = report['weighted avg']['recall']
        accuracy = report['accuracy']
        
        
        # Check if current model has the lowest validation loss
        if best_accuracy < accuracy:
            # Update best validation loss and hyperparameters
            best_accuracy = accuracy
            best_hyperparameters = {'learning_rate': lr, 'optimizer': optimizer_class.__name__}
            # Remove previous best model if exists
            if os.path.exists(best_model_path):
                os.remove(best_model_path)
                

Epoch 1/40


I0000 00:00:1711694315.691645      68 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


416/416 ━━━━━━━━━━━━━━━━━━━━ 174s 257ms/step - accuracy: 0.9442 - loss: 0.2211 - val_accuracy: 0.9738 - val_loss: 0.3196 - learning_rate: 0.0010
Epoch 2/40
416/416 ━━━━━━━━━━━━━━━━━━━━ 67s 160ms/step - accuracy: 0.9958 - loss: 0.0155 - val_accuracy: 0.9744 - val_loss: 0.1146 - learning_rate: 0.0010
Epoch 3/40
416/416 ━━━━━━━━━━━━━━━━━━━━ 65s 155ms/step - accuracy: 0.9950 - loss: 0.0269 - val_accuracy: 0.7616 - val_loss: 0.6367 - learning_rate: 0.0010
Epoch 4/40
416/416 ━━━━━━━━━━━━━━━━━━━━ 65s 157ms/step - accuracy: 0.9947 - loss: 0.0209 - val_accuracy: 0.9708 - val_loss: 0.1968 - learning_rate: 0.0010
Epoch 5/40
416/416 ━━━━━━━━━━━━━━━━━━━━ 67s 161ms/step - accuracy: 0.9970 - loss: 0.0089 - val_accuracy: 0.9964 - val_loss: 0.0233 - learning_rate: 2.0000e-04
Epoch 6/40
416/416 ━━━━━━━━━━━━━━━━━━━━ 66s 160ms/step - accuracy: 0.9989 - loss: 0.0029 - val_accuracy: 0.9961 - val_loss: 0.0203 - learning_rate: 2.0000e-04
Epoch 7/40
416/416 ━━━━━━━━━━━━━━━━━━━━ 65s 155ms/step - accuracy: 0.999

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/40
416/416 ━━━━━━━━━━━━━━━━━━━━ 106s 194ms/step - accuracy: 0.3289 - loss: 1.0993 - val_accuracy: 0.3307 - val_loss: 1.0986 - learning_rate: 0.1000
Epoch 2/40
416/416 ━━━━━━━━━━━━━━━━━━━━ 62s 149ms/step - accuracy: 0.3375 - loss: 1.0991 - val_accuracy: 0.3307 - val_loss: 1.0986 - learning_rate: 0.1000
Epoch 3/40
416/416 ━━━━━━━━━━━━━━━━━━━━ 62s 150ms/step - accuracy: 0.3346 - loss: 1.0992 - val_accuracy: 0.3250 - val_loss: 1.0989 - learning_rate: 0.1000
Epoch 4/40
416/416 ━━━━━━━━━━━━━━━━━━━━ 62s 149ms/step - accuracy: 0.3332 - loss: 1.0988 - val_accuracy: 0.3250 - val_loss: 1.0991 - learning_rate: 0.0200
Epoch 5/40
416/416 ━━━━━━━━━━━━━━━━━━━━ 62s 150ms/step - accuracy: 0.3361 - loss: 1.0988 - val_accuracy: 0.3250 - val_loss: 1.0990 - learning_rate: 0.0200
Epoch 6/40
416/416 ━━━━━━━━━━━━━━━━━━━━ 62s 149ms/step - accuracy: 0.3375 - loss: 1.0986 - val_accuracy: 0.3250 - val_loss: 1.0989 - learning_rate: 0.0040
Epoch 6: early stopping
Restoring model weights from the end of the b

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
print(best_hyperparameters)
print(best_accuracy)

{'learning_rate': 0.1, 'optimizer': 'Adam'}
0.7091969645486466


In [9]:
# import tensorflow as tf
# from sklearn.metrics import classification_report

# best_model = load_model(best_model_path)

# with h5py.File('/kaggle/input/selectedmitbih/test_dataset.h5', 'r') as file:
#     test_data = file['resized_dataset']
#     test_labels = file['beat_type']
#     test_data = np.array(test_data)
#     test_labels = np.array(test_labels)
    
# test_labels = to_categorical(test_labels, num_classes=3)

# print('start prediction')
# # Evaluate the model on the test data
# predictions = best_model.predict(test_data)
# predicted_labels = tf.argmax(predictions, axis=1)
# true_labels = tf.argmax(test_labels, axis=1)  # Assuming you have true labels for comparison
# print('prediction complete')

# # Calculate evaluation metrics
# report = classification_report(true_labels, predicted_labels, output_dict=True)

# f1_score = report['weighted avg']['f1-score']
# precision = report['weighted avg']['precision']
# positive_predictive_value = report['weighted avg']['precision']
# specificity = report['weighted avg']['recall']
# sensitivity = report['weighted avg']['recall']
# accuracy = report['accuracy']

# print(f"F1 Score: {f1_score}")
# print(f"Precision: {precision}")
# print(f"Positive Predictive Value: {positive_predictive_value}")
# print(f"Specificity: {specificity}")
# print(f"Sensitivity: {sensitivity}")
# print(f"Accuracy: {accuracy}")

In [10]:
# # Define optimizer with specified learning rate
# learning_rate = 0.001
# optimizer = Adam(learning_rate=learning_rate)

# # Compile the model with the optimizer and loss function
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# # Add callbacks for learning rate reduction and early stopping based on validation loss
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)
# early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# # Fit the data to the model
# history = model.fit(train_data, train_labels, batch_size=32, epochs=40, validation_data=(val_data, val_labels), callbacks=[reduce_lr, early_stop])

In [11]:
# # Access the best model
# model = grid_result.best_estimator_.model

# # Save the best model
# model.save('best_model.h5')

In [12]:
# !zip -r file.zip /kaggle/working

In [13]:
# !ls

In [14]:
# from IPython.display import FileLink
# FileLink(r'file.zip')

In [15]:
# import tensorflow as tf
# from sklearn.metrics import classification_report
# print('Read dataset complete')

# with h5py.File('/kaggle/input/selectedmitbih/test_dataset.h5', 'r') as file:
#     test_data = file['resized_dataset']
#     test_labels = file['beat_type']
#     test_data = np.array(test_data)
#     test_labels = np.array(test_labels)
    
# test_labels = to_categorical(test_labels, num_classes=3)

# print('start prediction')
# # Evaluate the model on the test data
# predictions = model.predict(test_data)
# predicted_labels = tf.argmax(predictions, axis=1)
# true_labels = tf.argmax(test_labels, axis=1)  # Assuming you have true labels for comparison
# print('prediction complete')

# # Calculate evaluation metrics
# report = classification_report(true_labels, predicted_labels, output_dict=True)

# f1_score = report['weighted avg']['f1-score']
# precision = report['weighted avg']['precision']
# positive_predictive_value = report['weighted avg']['precision']
# specificity = report['weighted avg']['recall']
# sensitivity = report['weighted avg']['recall']
# accuracy = report['accuracy']

# print(f"F1 Score: {f1_score}")
# print(f"Precision: {precision}")
# print(f"Positive Predictive Value: {positive_predictive_value}")
# print(f"Specificity: {specificity}")
# print(f"Sensitivity: {sensitivity}")
# print(f"Accuracy: {accuracy}")